# Chapter9.SparkSQL

### 9.1连接SparkSQL
- 1.可选 使用带有HIVE支持的SparkSQL  需要拷贝 hive-site.xml到$Spark_HOME/conf 如果没部署好Hive则SsparkSQL会自己创建Hive元数据仓库

- 2.可选 文件系统 默认使用HDFS,需要拷贝hdfs-site.xml到$Spark_HOME/conf 没有则使用本地文件系统

### 9.2在Applicaition中使用SparkSQL


#### 初始化SparkSQL

In [1]:
from pyspark.sql import HiveContext,Row
from pyspark.sql import SQLContext,Row
from pyspark import SparkConf,SparkContext
sc = SparkContext("local[*]","HiveTest")
hiveCtx = HiveContext(sc)

#### 创建表测试

In [15]:
hiveCtx.sql("""
create table page_view  
(  
page_id bigint comment '页面ID',  
page_name string comment '页面名称',  
page_url string comment '页面URL'  
)  
""")

DataFrame[result: string]

In [22]:
hiveCtx.sql("insert into page_view values(1,'Niko','www.baidu.com')")

AnalysisException: "\nUnsupported language features in query: insert into page_view values(1,'Niko','www.baidu.com')\nTOK_QUERY 0, 0,13, 0\n  TOK_FROM 0, -1,13, 0\n    TOK_VIRTUAL_TABLE 0, -1,13, 0\n      TOK_VIRTUAL_TABREF 0, -1,-1, 0\n        TOK_ANONYMOUS 0, -1,-1, 0\n      TOK_VALUES_TABLE 1, 6,13, 29\n        TOK_VALUE_ROW 1, 7,13, 29\n          1 1, 8,8, 29\n          'Niko' 1, 10,10, 31\n          'www.baidu.com' 1, 12,12, 38\n  TOK_INSERT 1, 0,-1, 12\n    TOK_INSERT_INTO 1, 0,4, 12\n      TOK_TAB 1, 4,4, 12\n        TOK_TABNAME 1, 4,4, 12\n          page_view 1, 4,4, 12\n    TOK_SELECT 0, -1,-1, 0\n      TOK_SELEXPR 0, -1,-1, 0\n        TOK_ALLCOLREF 0, -1,-1, 0\n\nscala.NotImplementedError: No parse rules for:\n TOK_VIRTUAL_TABLE 0, -1,13, 0\n  TOK_VIRTUAL_TABREF 0, -1,-1, 0\n    TOK_ANONYMOUS 0, -1,-1, 0\n  TOK_VALUES_TABLE 1, 6,13, 29\n    TOK_VALUE_ROW 1, 7,13, 29\n      1 1, 8,8, 29\n      'Niko' 1, 10,10, 31\n      'www.baidu.com' 1, 12,12, 38\n \norg.apache.spark.sql.hiv

#### 基本查询

In [33]:
inputFile = "file:///Users/lixiwei-mac/Documents/git/learning-spark/files/testweet.json"
input = hiveCtx.jsonFile(inputFile)
# 注册输入的SchemaRDD(现在更名为DataFrame)
input.registerTempTable("tweets")
# 依据retweetCount选出推文
topTweets = hiveCtx.sql("select text,retweetCount from tweets order by retweetCount limit 10")
print(input.printSchema())
print(topTweets.collect())

root
 |-- contributorsIDs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- createdAt: string (nullable = true)
 |-- currentUserRetweetId: long (nullable = true)
 |-- hashtagEntities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- inReplyToStatusId: long (nullable = true)
 |-- inReplyToUserId: long (nullable = true)
 |-- isFavorited: boolean (nullable = true)
 |-- isPossiblySensitive: boolean (nullable = true)
 |-- isTruncated: boolean (nullable = true)
 |-- mediaEntities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- retweetCount: long (nullable = true)
 |-- source: string (nullable = true)
 |-- text: string (nullable = true)
 |-- urlEntities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- user: struct (nullable = true)
 |    |-- createdAt: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- descriptionURL

/Users/lixiwei-mac/app/spark-1.6.0-bin-hadoop2.6/python/pyspark/sql/context.py:481: UserWarning: jsonFile is deprecated. Use read.json() instead.
  warnings.warn("jsonFile is deprecated. Use read.json() instead.")


#### SchemaRDD

In [25]:
topTweetText = topTweets.map(lambda row:row.text)
topTweetText.collect()

['Adventures With Coffee, Code, and Writing.']

#### 缓存

In [31]:
hiveCtx.cacheTable("tweets")
topTweetText.persist()

PythonRDD[51] at collect at <ipython-input-25-cd7d9a574eee>:2

### 9.3 读取和存储数据

In [32]:
rows = hiveCtx.sql("select key , value from mytable")
keys = rows.map(lambda row : row[0])

AnalysisException: 'Table not found: mytable; line 1 pos 24'

#### Json

In [36]:
rows = hiveCtx.sql("select user. followersCount from tweets")
print(rows.collect())

[Row(followersCount=1231)]


#### 基于RDD

In [41]:
user = sc.parallelize([Row(name="NikoBelic",age="18")])
userDataFrame = hiveCtx.inferSchema(user)
userDataFrame.registerTempTable("user")

res = hiveCtx.sql("select * from user")
res.collect()

/Users/lixiwei-mac/app/spark-1.6.0-bin-hadoop2.6/python/pyspark/sql/context.py:286: UserWarning: inferSchema is deprecated, please use createDataFrame instead.
  warnings.warn("inferSchema is deprecated, please use createDataFrame instead.")
/Users/lixiwei-mac/app/spark-1.6.0-bin-hadoop2.6/python/pyspark/sql/dataframe.py:142: UserWarning: Use registerTempTable instead of registerAsTable.
  warnings.warn("Use registerTempTable instead of registerAsTable.")


[Row(age='18', name='NikoBelic')]

### 9.4 JDBC/ODBC 服务器
- start-thriftserver.sh
- sudo beeline -u jdbc:hive2://localhost:10000


### 9.5 用户自定义函数UDF

In [56]:
# 求推文长度
from pyspark.sql.types import IntegerType
hiveCtx.registerFunction("strLenPython",lambda x:len(x),IntegerType())
lengthSchemaRDD = hiveCtx.sql("select strLenPython(text) from tweets limit 10")
lengthSchemaRDD.collect()

[Row(_c0=42)]

### 9.6 SprakSQL性能